Current example follows this [document](https://pytorch.org/tutorials/beginner/basics/optimization_tutorial.html) .

In [1]:
# 1. Prepare dependencies
import mnist

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn


In [2]:
# 2. prepare training and testing dataloader

mnist.datasets_url="http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/"

train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()


train_images_tensor = torch.Tensor(train_images)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.int64)
test_images_tensor = torch.Tensor(test_images)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.int64)

train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)


train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)
train_images.shape

(60000, 28, 28)

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
            nn.Linear(10, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
model

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=10, bias=True)
  )
)

In [4]:
# Define hyperparameters

learning_rate = 1e-3
batch_size = 64

epochs = 50


In [5]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

# Define the train_loop function

def train_loop(dataloader, model, loss_fn, optimizer, epoch):
    
    model.train()

    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        
        
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        writer.add_scalar("Loss/train", loss.item(), int(60000/batch_size)*epoch+batch)



def test_loop(dataloader, model, loss_fn):
    
    model.eval()

    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y ).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /=size

    print(f'Test Error: \n Accurary: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n')






In [6]:
loss_fn = nn.CrossEntropyLoss()


optimizer =torch.optim.SGD(model.parameters(), lr=learning_rate)




for t in range(epochs):
    # print(f'Epoch {t+1}\n--------------------')
    loss = train_loop(train_dataloader, model, loss_fn, optimizer, t)
    test_loop(test_dataloader, model, loss_fn)

writer.flush()
writer.close()


Test Error: 
 Accurary: 18.2%, Avg loss: 2.121914 

Test Error: 
 Accurary: 18.6%, Avg loss: 2.104190 

Test Error: 
 Accurary: 19.3%, Avg loss: 2.079617 

Test Error: 
 Accurary: 19.5%, Avg loss: 2.069535 

Test Error: 
 Accurary: 19.9%, Avg loss: 2.060065 

Test Error: 
 Accurary: 21.5%, Avg loss: 1.956375 

Test Error: 
 Accurary: 23.0%, Avg loss: 1.862645 

Test Error: 
 Accurary: 36.0%, Avg loss: 1.617708 

Test Error: 
 Accurary: 38.9%, Avg loss: 1.461745 

Test Error: 
 Accurary: 42.1%, Avg loss: 1.401692 

Test Error: 
 Accurary: 44.1%, Avg loss: 1.386905 

Test Error: 
 Accurary: 48.0%, Avg loss: 1.167336 

Test Error: 
 Accurary: 63.8%, Avg loss: 1.027813 

Test Error: 
 Accurary: 64.2%, Avg loss: 0.900119 

Test Error: 
 Accurary: 73.7%, Avg loss: 0.844037 

Test Error: 
 Accurary: 74.5%, Avg loss: 0.793071 

Test Error: 
 Accurary: 67.8%, Avg loss: 0.870554 

Test Error: 
 Accurary: 74.8%, Avg loss: 0.718014 

Test Error: 
 Accurary: 77.2%, Avg loss: 0.708958 

Test Error: 